In [1]:
%pip install pymupdf
%pip install langdetect

Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 15.7 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=5562828d5db582afbe799f3315aaa32c5fd7e68835aecedd230ba4897927b3f1
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect
Note: you may need to restart the kernel to use updated packages.


In [54]:
from pathlib import Path
import pymupdf
import json
from langdetect import detect
from tqdm.auto import tqdm
from typing import List
from docling.datamodel.document import DoclingDocument

In [69]:
# TODO: 
# - find number of embedded images
# - docling
# - pdfplumber
# - spacy
# - nltk

class Metadata():
    def __init__(self):
        pass

    def ocr_text_statistics(self, file_name: str, ocr_folder: Path, file_type: str = "md"):
        ocr_text = (ocr_folder / f"{file_name}.{file_type}").read_text()

        statistics = {
            "num_words": len(ocr_text.split()),
            "num_lines": len(ocr_text.splitlines()),
            "num_chars": len(ocr_text),
        }

        return statistics

    def language_metadata(self, file_name: str, ocr_folder: Path, file_type: str = "md"):
        ocr_text = (ocr_folder / f"{file_name}.{file_type}").read_text()

        try:
            language = detect(ocr_text)
        except:
            language = "unknown"

        metadata = {
            "language": language
        }

        return metadata

    def pdf_statistics(self, file_name: str, pdf_folder: Path):
        doc = pymupdf.open(pdf_folder / f"{file_name}.pdf")

        metadata = {
            "num_pages": doc.page_count,
            "form_fields": doc.is_form_pdf,
            "table_of_contents": doc.get_toc(simple=True),
            **doc.metadata
        }

        return metadata
    
    def docling_metadata(self, file_name: str, ocr_folder: Path, file_type: str = "json"):
        docling_path = ocr_folder / f"{file_name}.{file_type}"
        docling_json = DoclingDocument.load_from_json(docling_path)

        metadata = {
            "section_headers": [t.text for t in docling_json.texts if t.label == "section_header"],
            "has_tables": len(docling_json.tables) > 0
        }

        return metadata

    def process_file(self, file_name: str, ocr_folder: Path, pdf_folder: Path, result_path: Path):
        metadata = {}

        if result_path.exists():
            metadata = json.load(result_path.open())

        # only add metadata if it is not already present => enable future addition of new metadata
        if "ocr_text_statistics" not in metadata:
            metadata["ocr_text_statistics"] = self.ocr_text_statistics(file_name, ocr_folder)
        if "pdf_statistics" not in metadata:
            metadata["pdf_statistics"] = self.pdf_statistics(file_name, pdf_folder)
        if "language_metadata" not in metadata:
            metadata["language_metadata"] = self.language_metadata(file_name, ocr_folder)
        if "docling_metadata" not in metadata:
            metadata["docling_metadata"] = self.docling_metadata(file_name, ocr_folder)

        return metadata

    def process_metadata(self, pdf_folder: Path, ocr_folder: Path, results_folder: Path = Path("metadata"), file_type: str = "md"):
        ocr_files = list(ocr_folder.glob(f"*.{file_type}")) # use ocr files as baseline as some pdfs were not able to be processed
        file_stems = [file.stem for file in ocr_files]

        results_folder.mkdir(parents=True, exist_ok=True)
        
        for stem in tqdm(file_stems, desc="Processing metadata"):
            result_path = results_folder / f"{stem}.json"
            metadata = self.process_file(stem, ocr_folder, pdf_folder, result_path)
            json.dump(metadata, result_path.open("w"), ensure_ascii=False, indent=4)


In [70]:
metadata = Metadata()

In [71]:
metadata.process_metadata(Path("pdfs"), Path("docling"))

Processing metadata:   0%|          | 0/50718 [00:00<?, ?it/s]

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

MuPDF error: format error: object out of range (11 0 R); xref size 11

MuPDF error: format error: object out of range (14 0 R); xref size 11

MuPDF error: format error: object out of range (12 0 R); xref size 11

MuPDF error: format error: object out of range (13 0 R); xref size 11

MuPDF error: format error: object out of range (17 0 R); xref size 17

MuPDF error: format error: object out of range (17 0 R); xref size 17

MuPDF error: format error: object out of range (17 0 R); xref size 17

MuPDF error: format error: object out of range (17 0 R); xref size 17

